In [33]:
import folium
import pandas as pd
import gpxpy
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib as mpl
import numpy as np
import collections
%matplotlib notebook


In [41]:
bzn  = gpxpy.parse(open('activity_4257789098.gpx')) # BZN
sf   = gpxpy.parse(open('activity_4334092284.gpx')) # SF
kc   = gpxpy.parse(open('activity_4292449432.gpx')) # KC
bzn2 = gpxpy.parse(open('activity_4264388805.gpx')) # BZN2


gpx = bzn2
track = gpx.tracks[0]
segment = track.segments[0]

In [42]:
data = []
segment_length = segment.length_3d()

extension_data = collections.defaultdict(list)
for point_idx, point in enumerate(segment.points):
    extensions = point.extensions

    for child in extensions[0].getchildren(): # this gets the extension data (hr and cadence) from the gpx file
        tag = child.tag.rsplit("}", 1)[-1]
        value = child.text
        extension_data[tag].append(float(value))
            

    data.append([point.longitude, point.latitude,
                 point.elevation, point.time, segment.get_speed(point_idx)])
    

columns = ['Longitude', 'Latitude', 'Altitude', 'Time', 'Speed']
df = pd.DataFrame(data, columns=columns)
df['points'] = list(zip(df.Latitude, df.Longitude))

df.Time = pd.to_datetime(df.Time, format = "%Y-%m-%d %H:%M:%S")
df.Time = df.Time.dt.tz_localize(None)
df.Time = (df.Time- df.Time.values[0]).dt.total_seconds()

df['hr'] = extension_data['hr']
df['cad'] = extension_data['cad']
# df['hr'] = df.hr.rolling(25, center = True).mean() # smooth the heart rate
# df = df.dropna()
df.head()

,Longitude,Latitude,Altitude,Time,Speed,points,hr,cad
0,-111.009164,45.672395,1489.800049,0.0,1.519856,"(45.672394847497344, -111.00916385650635)",101.0,58.0
1,-111.009200,45.672364,1488.599976,3.0,1.546610,"(45.67236425355077, -111.00919964723289)",103.0,59.0
2,-111.009250,45.672343,1487.400024,6.0,2.097733,"(45.67234321497381, -111.00925002247095)",102.0,59.0
3,-111.009308,45.672363,1488.800049,8.0,2.926911,"(45.672363080084324, -111.00930844433606)",105.0,122.0
4,-111.009377,45.672394,1487.800049,10.0,3.540358,"(45.67239417694509, -111.00937742739916)",104.0,80.0


### Let's re-calculate the speed because I'm not convinced

In [43]:
def running_speed(time, lat, lon, alt):
    """
    Calculates your running speed given a time series of 
    time, lat, lon, and alt coordinates.

    Units of velocity is meters/sec.
    """
    dt = [(ti-tj) for ti, tj in zip(time[1:], time[:-1])]

    X1 = np.array([lat[1:], lon[1:], alt[1:]]).T
    X2 = np.array([lat[:-1], lon[:-1], alt[:-1]]).T
    dx = 1000*haversine(X1, X2) # In units of meters
    
    return np.append([0], dx/dt)

def haversine(X1, X2):
    """
    I know that haversine is overkill...

    Implementation of the haversine foruma to calculate total distance
    at an average altitude. X1 and X2 must be N*3 array of 
    lat, lon, alt. Units of alt is km.
    """
    Re_km = 6371
    R = (Re_km+(X1[:, 2]+X2[:, 2])/2) # Mean distance from Earth's center.
    s = 2*np.arcsin( np.sqrt( np.sin(np.deg2rad(X1[:, 0]-X2[:, 0])/2)**2 + \
                    np.cos(np.deg2rad(X1[:, 0]))*np.cos(np.deg2rad(X2[:, 0]))*\
                    np.sin(np.deg2rad(X1[:, 1]-X2[:, 1])/2)**2 ))
    return R*s

In [44]:
df.Speed = running_speed(df.Time, df.Latitude, df.Longitude, df.Altitude)*2.23694 # convert to mph

In [45]:
bzn = [45.6770, -111.0429]

loc = [df.Latitude.mean(), df.Longitude.mean()]



m = folium.Map(location = loc,zoom_start=14)
def rgb2hex(rgb):
    rgb = [hex(int(256*x)) for x in rgb]
    r, g, b = [str(x)[2:] for x in rgb]
    
    if r == '100':
        r = 'ff'
    if g == '100':
        g = 'ff'
    if b == '100':
        b= 'ff'
#     print(r, g, b)
    return f"#{r.zfill(2)}{g.zfill(2)}{b.zfill(2)}ff"

norm = mpl.colors.Normalize(vmin=0, vmax=max(df.Speed.values)+1)
color_mapper = cm.ScalarMappable(norm = norm,cmap=cm.jet)
rgb_vals = []
for v in df.Speed.values:
    rgb_vals.append(list(color_mapper.to_rgba(v)[0:3]))

colors = [rgb2hex(rgb) for rgb in rgb_vals]

df['Colors'] = colors


In [46]:
for i, row in df.iterrows():
    folium.CircleMarker(location=row.points,
                        radius=5,
                        popup=str(row.Speed),
                        color=row.Colors, fill = True).add_to(m)
    

m    
